In [1]:
import requests
import bs4
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from uszipcode import SearchEngine

In [2]:
job_title = "zoo keeper"
url = f'https://www.indeed.com/jobs?q={job_title}&limit=50'

In [3]:
def scrapeData(class_str, list_name):
    element = soup.find_all(class_=class_str)
    for i in element:
        list_name.append(i.text.strip())

In [4]:
def scrapeSalary(num, list_name):
    salary_tags = soup.find_all(class_='row')
    for i in range(len(salary_tags)):
        try:
            salary = salary_tags[i].find(class_='salary').text.strip()
            list_name.append(salary)
        except:
            list_name.append("No Salary Info Provided")

In [5]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser')

In [6]:
job_titles = []
company_names = []
locations = []
salaries = []

In [7]:
scrapeData('title', job_titles)
scrapeData('company', company_names)
scrapeData('location', locations)
num = len(job_titles)
scrapeSalary(num, salaries)

In [8]:
url2 = f'https://www.indeed.com/jobs?q={job_title}&limit=50'
browser.visit(url2)
html = browser.html
soup2 = bs(html, 'html.parser')
browser.quit()

In [9]:
scrapeData('title', job_titles)
scrapeData('company', company_names)
scrapeData('location', locations)
num = len(job_titles)
scrapeSalary(num, salaries)

In [10]:
def hasZip(inputString):
    if any(char.isdigit() for char in inputString):
        #return inputString[-5:]
        return "".join(filter(lambda x: x.isdigit(), inputString))
    else:
        return inputString

In [11]:
loc_list = []
for i in locations:
   loc_list.append(hasZip(i))

In [12]:
## ADD CODE HERE TO CONVERT CITY TO ZIP
## THEN CONVERT ZIP TO LAT, LNG

In [13]:
def getZip(str, zipList):
    if str.isdigit():
        zipList.append(str)
    else:
        try:
            search = SearchEngine(simple_zipcode=True)
            city_state_str = str.split(', ')
            city_state = search.by_city_and_state(city_state_str[0], city_state_str[1])
            zipList.append(city_state[0].zipcode)
        except:
            zipList.append("None")
            print(f"No City, State info found for {str}")

In [14]:
zip_list = []

for i in loc_list:
    getZip(i, zip_list)

In [15]:
def getCoord(str, coordList, cityList):
    search = SearchEngine(simple_zipcode=True)
    if str.isdigit():
        try:
            zipcode = search.by_zipcode(str)
            coords = [zipcode.lat, zipcode.lng]
            coordList.append(coords)
            cityList.append(zipcode.post_office_city)
        except:
            coordList.append(["None", "None"])
            cityList.append("None")
            print(f"No Results Found for Zip {str}")
    else:
        coordList.append(["None", "None"])
        cityList.append(str)

In [16]:
search_limit = 100

In [17]:
coord_list = []
city_list = []

for i in zip_list[0:search_limit]:
    getCoord(i, coord_list, city_list)

In [18]:
job_list = []

for i in range(search_limit):
    job_dict = {}
    job_dict['Title'] = job_titles[i]
    job_dict['Company'] = company_names[i]
    job_dict['Location'] = city_list[i]
    job_dict["Coordinates"] = coord_list[i]
    job_dict["Salary_Info"] = salaries[i]
    job_list.append(job_dict)

In [19]:
for i in job_list:
    #print(i['Coordinates'])
    if "None" in str(i["Coordinates"]):
        job_list.remove(i)

In [20]:
len(job_list)

100

In [21]:
len(coord_list)

100

In [22]:
job_list

[{'Title': 'Aquarium Maintenance Specialist - Kansas City',
  'Company': 'Reef Tectonics Inc',
  'Location': 'Kansas City, MO',
  'Coordinates': [39.104, -94.599],
  'Salary_Info': '$14 - $15 an hour'},
 {'Title': 'Professional Dog Trainer',
  'Company': 'Wag Hotels',
  'Location': 'Redwood City, CA',
  'Coordinates': [37.46, -122.24],
  'Salary_Info': '$15 - $18 an hour'},
 {'Title': '(Physician) Orthopedic Surgery: Spine - Orthopedic Spine Sur...',
  'Company': 'Arlington HealthCare',
  'Location': 'Wichita, KS',
  'Coordinates': [37.686, -97.335],
  'Salary_Info': 'No Salary Info Provided'},
 {'Title': 'Animal Keeper - Regular, Full-Time - Hoofstock',
  'Company': 'San Francisco Zoo',
  'Location': 'San Francisco, CA',
  'Coordinates': [37.72, -122.48],
  'Salary_Info': 'No Salary Info Provided'},
 {'Title': 'Keeper I – Equine Instructor',
  'Company': 'Phoenix Zoo',
  'Location': 'Phoenix, AZ',
  'Coordinates': [33.46, -111.99],
  'Salary_Info': 'No Salary Info Provided'},
 {'Title